In [1]:
# import necessary library
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
# import csv dataset
file = "tubes2_HeartDisease_train.csv"
df = pd.read_csv(file)

In [3]:
# split feature and label
feature = df.drop("Column14",inplace=False,axis=1)
label = df["Column14"]

In [4]:
# handle missing value

header = feature.columns.values.tolist()
feature_impute = feature.replace('?',np.nan)

imputer_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

discrete_value = ['Column1','Column2','Column3','Column6','Column7','Column9','Column13']
continues_value = ['Column4','Column5','Column8','Column10','Column11','Column12']

imputer_mode.fit(feature_impute[discrete_value])
feature_impute[discrete_value] = imputer_mode.transform(feature_impute[discrete_value])

imputer_mean.fit(feature_impute[continues_value])
feature_impute[continues_value] = imputer_mean.transform(feature_impute[continues_value])

feature_impute['Column13'] = pd.to_numeric(feature_impute['Column13'])
feature_impute.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13
0,54,1,4,125.0,216.0,0,0,140.0,0,0.0,1.762089,0.686792,3
1,55,1,4,158.0,217.0,0,0,110.0,1,2.5,2.000000,0.686792,3
2,54,0,3,135.0,304.0,1,0,170.0,0,0.0,1.000000,0.000000,3
3,48,0,3,120.0,195.0,0,0,125.0,0,0.0,1.762089,0.686792,3
4,50,1,4,120.0,0.0,0,1,156.0,1,0.0,1.000000,0.686792,6


In [5]:
idx_to_drop = []

for item in continues_value:
    mean = feature_impute[item].mean()
    std = feature_impute[item].std()
    low_threshold = mean - 2*std
    high_threshold = mean + 2*std
    
#     print(item, low_threshold, high_threshold)
    
    for i in range(feature_impute[item].shape[0]):
        cur_value = feature_impute[item].iloc[i]
        if (cur_value < low_threshold or cur_value > high_threshold):
            idx_to_drop.append(i)

feature_impute.drop(feature_impute.index[idx_to_drop],inplace=True)
label.drop(label.index[idx_to_drop],inplace=True)
# feature_impute.describe()

In [6]:
# feature scale
feature_scale = pd.DataFrame(preprocessing.scale(feature_impute), columns=header)
feature_scale.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by the scale function.
  


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13
0,0.176725,0.555458,0.852692,-0.380985,0.149048,-0.387298,-0.693683,-0.015270,-0.700908,-0.567213,0.253637,0.396182,-0.510509
1,0.285116,0.555458,0.852692,1.952471,0.158792,-0.387298,-0.693683,-1.299403,1.426721,0.036852,0.867040,0.396182,-0.510509
2,0.176725,-1.800315,-0.236859,0.326123,1.006587,2.581989,-0.693683,1.268864,-0.700908,-0.567213,-1.711246,-1.883308,-0.510509
3,-0.473623,-1.800315,-0.236859,-0.734539,-0.055593,-0.387298,-0.693683,-0.657336,-0.700908,-0.567213,0.253637,0.396182,-0.510509
4,-0.256840,0.555458,0.852692,-0.734539,-1.955823,-0.387298,0.619047,0.669601,1.426721,-0.567213,-1.711246,0.396182,1.417604


In [7]:
# display feature correlation with label
for feat in feature_scale:
    print(feat, feature_scale[feat].corr(label))

Column1 0.00950298929003149
Column2 0.002169923491672284
Column3 -0.0019815812945992656
Column4 -0.06693641003884788
Column5 -0.11776403090762269
Column6 -0.03524705294950225
Column7 -0.053834154188509495
Column8 0.029848809975586067
Column9 -0.05664339720406514
Column10 -0.08849686660664915
Column11 -0.07955841604138246
Column12 -0.023649597038712267
Column13 0.061955162333051726


In [8]:
# drop feature with low correlation
# feature_scale.drop("Column4",axis=1,inplace=True)
# feature_scale.drop("Column6",axis=1,inplace=True)
# feature_scale.drop("Column7",axis=1,inplace=True)

import itertools
feature_combinations = []

# print(list(itertools.combinations(header,1)))
for i in range(1,14):
    feature_combinations.append(list(itertools.combinations(header,i)))

In [15]:
kf = KFold(n_splits=10)

best_model = None
best_accuracy = 0.0

best_header = None

i = 6

for j in range(len(feature_combinations[i])):
    cur_header = []
    for k in range(len(feature_combinations[i][j])):
        cur_header.append(feature_combinations[i][j][k])

    cur_feature = feature_scale[cur_header]
    for train_idx,test_idx in kf.split(cur_feature):
        X_train = cur_feature.iloc[train_idx] 
        y_train = label.iloc[train_idx]       

        X_test = cur_feature.iloc[test_idx] 
        y_test = label.iloc[test_idx] 

        cur_model = tree.DecisionTreeClassifier(
            criterion='entropy', 
            min_samples_leaf=34, 
            max_depth=5, 
        ).fit(X_train, y_train)
        
        y_predict = cur_model.predict(X_test)
        cur_accuracy = accuracy_score(y_test, y_predict)

        if (cur_accuracy > best_accuracy):
            best_model = cur_model
            best_accuracy = cur_accuracy
            best_header = cur_header
    
print(best_header)
print(best_accuracy)

['Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column8', 'Column9']
0.7333333333333333
